In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# change root dit
import os
def get_root_dir():
    if os.path.exists('/content/drive/MyDrive/'):
        return '/content/drive/MyDrive/AMEX Project/notebooks' 
    else:
        return './' 

os.chdir(get_root_dir())

Load data

In [4]:
train_labels = pd.read_csv('../data/1-original-data/train_labels.csv')
print(train_labels.shape)
train_labels.head(2)

(458913, 2)


,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0


In [5]:
train_df = pd.read_parquet('../data/1-original-data/train.parquet', engine = 'pyarrow')
print(train_df.shape)
train_df.head(2)

(5531451, 190)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,0,0,0.0,NaN,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,0,0,0.0,NaN,0,0.005492,0


Drop customers with fewer than 13 records

In [6]:
records_count = train_df.groupby('customer_ID').count()
print(records_count.shape)
records_count.head(2)

(458913, 189)


,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,13,13,13,13,13,13,13,13,13,0,...,13,13,13,13,13,13,0,13,13,13
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,13,13,13,13,13,13,13,13,13,0,...,13,13,13,13,13,13,0,13,13,13


In [7]:
to_keep = records_count[records_count['S_2'] == 13].index.tolist()
print(len(to_keep))

386034


In [8]:
train_df = train_df[train_df['customer_ID'].isin(to_keep)]
train_labels = train_labels[train_labels['customer_ID'].isin(to_keep)]
print(train_df.shape, train_labels.shape)

(5018442, 190) (386034, 2)


Create test set

In [ ]:
train_labels['target'].value_counts()

0    296556
1     89478
Name: target, dtype: int64

In [9]:
train_ids, test_ids = train_test_split(train_labels, test_size = 0.05, random_state = 2022, stratify = train_labels['target'])

In [ ]:
print(test_ids.shape)
test_ids['target'].value_counts()

(19302, 2)


0    14828
1     4474
Name: target, dtype: int64

In [ ]:
test_df = train_df[train_df['customer_ID'].isin(test_ids['customer_ID'].tolist())]
print(test_df.shape)

(250926, 190)


In [ ]:
test_df = test_df.merge(test_ids, on='customer_ID', how='left')
print(test_df.shape)
test_df.head(2)

(250926, 191)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,00062d39ff054c4e18d5dba3b988c94277246d2d4aee42...,2017-03-22,0.749412,0,0.007876,1.001364,0.003513,1.045401,0.0,0.001397,...,-1,-1,0,0,0.0,NaN,0,0.009471,0,0
1,00062d39ff054c4e18d5dba3b988c94277246d2d4aee42...,2017-04-02,0.743547,4,0.013650,1.006042,0.004655,0.963630,0.0,0.009010,...,-1,-1,0,0,0.0,NaN,0,0.000094,0,0


In [ ]:
test_ids.to_csv('../data/9-BT4240-data/test_labels.csv', index=False)

In [ ]:
test_df.to_parquet('../data/9-BT4240-data/test.parquet', engine='pyarrow', index=False)

In [ ]:
del test_df

Sample train set

In [13]:
print(train_ids.shape)

(366732, 2)


In [24]:
train_others, train_samp = train_test_split(train_ids, test_size=0.12, random_state=2022, stratify=train_ids['target'])
print(train_samp.shape)

(44008, 2)


In [25]:
train_samp['target'].value_counts() / train_samp['target'].value_counts().sum()

0    0.768201
1    0.231799
Name: target, dtype: float64

In [26]:
train_keep = train_df[train_df['customer_ID'].isin(train_samp['customer_ID'].tolist())]
train_keep = train_keep.merge(train_samp, on='customer_ID', how='left')
print(train_keep.shape)
train_keep.head(2)

(572104, 191)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,0,0,0.0,NaN,0,0.000610,0,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,0,0,0.0,NaN,0,0.005492,0,0


In [27]:
train_keep.to_parquet('../data/train.parquet', engine='pyarrow', index=False)

In [28]:
train_samp.to_csv('../data/train_labels.csv', index=False)

Create balanced train set

In [12]:
train_ids['target'].value_counts() / train_ids['target'].value_counts().sum()

0    0.768212
1    0.231788
Name: target, dtype: float64

In [ ]:
# sample n = 10000 for each target type
train_samp = train_ids.groupby('target').sample(n=10000, random_state=2022)
print(train_samp.shape)
train_samp.head(2)

(20000, 2)


,customer_ID,target
271671,97497032ea2afacce595525ed09486d2a76ffd094c7e4a...,0
427573,ee61e342bbf1b71c57a91b61541267ab057861728ce8ce...,0


In [ ]:
train_keep = train_df[train_df['customer_ID'].isin(train_samp['customer_ID'].tolist())]
train_keep = train_keep.merge(train_samp, on='customer_ID', how='left')
print(train_keep.shape)
train_keep.head(2)

(260000, 191)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,00055add5eaee481b27e40af3f47b8b24e82c1e550f6ab...,2017-03-17,0.841486,0,0.002458,0.819486,0.007345,0.154343,0.0,0.011857,...,-1,-1,0,0,0.0,NaN,0,0.008964,0,0
1,00055add5eaee481b27e40af3f47b8b24e82c1e550f6ab...,2017-04-15,0.832555,0,0.005438,0.816720,0.001208,0.158774,0.0,0.015860,...,-1,-1,0,0,0.0,NaN,0,0.000613,0,0


In [ ]:
train_keep.to_parquet('../data/9-BT4240-data/train.parquet', engine='pyarrow', index=False)

In [ ]:
del train_keep

In [ ]:
gc.collect()

112

In [ ]:
train_samp.to_csv('../data/9-BT4240-data/train_labels.csv', index=False)

Save ununsed training data

In [ ]:
train_unused_ids = train_ids[~train_ids['customer_ID'].isin(train_samp['customer_ID'].tolist())]
print(train_unused_ids.shape)

(346732, 2)


In [ ]:
train_df = train_df[train_df['customer_ID'].isin(train_unused_ids['customer_ID'].tolist())]
train_df = train_df.merge(train_unused_ids, on='customer_ID', how='left')
print(train_df.shape)
train_df.head(2)

(4507516, 191)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,0,0,0.0,NaN,0,0.000610,0,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,0,0,0.0,NaN,0,0.005492,0,0


In [ ]:
train_unused_ids.to_csv('../data/9-BT4240-data/train_unused_labels.csv', index=False)

In [ ]:
train_df.to_parquet('../data/9-BT4240-data/train_unused.parquet', engine='pyarrow', index=False)